<a href="https://colab.research.google.com/github/csch7/CSCI-4170/blob/main/Homework-05/CNNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import torchvision
import torch

from google.colab import drive

drive.mount('/content/drive')

im_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),torchvision.transforms.ToTensor(),
                                               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Train/Train', transform = im_transform)
train_loader = torch.utils.data.DataLoader(train_dat, batch_size = 32, shuffle = True)
valid_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Validation/Validation', transform = im_transform)
valid_loader = torch.utils.data.DataLoader(valid_dat, batch_size=len(valid_dat), shuffle = True)
test_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Test/Test', transform = im_transform)
test_loader = torch.utils.data.DataLoader(test_dat, batch_size = len(test_dat))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
from torch import nn

class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride = 1, padding = 1)
    self.conv2 = nn.Conv2d(32, 32, kernel_size=(3,3), stride = 1, padding = 1)
    self.conv3 = nn.Conv2d(32, 16, kernel_size=(3,3), stride = 1, padding = 1)
    self.pool = nn.MaxPool2d((2,2))
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(16*32*32, 1024)
    self.fc2 = nn.Linear(1024, 128)
    self.fc3 = nn.Linear(128, 3)
    self.sm = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.pool(self.relu(self.conv1(x)))
    x = self.pool(self.relu(self.conv2(x)))
    x = self.pool(self.relu(self.conv3(x)))
    x = torch.flatten(x, 1)
    x = self.relu(self.fc1(x))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    return self.sm(x)




In [4]:
from torch import optim
from sklearn.metrics import accuracy_score
from tqdm import tqdm

epochs = 30
lr = 0.001
model = CNN()
# model.to(torch.device('cuda'))
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

for e in range(epochs):
  model.train()
  losses = []
  for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
    optimizer.zero_grad()
    # x = x.to(torch.device('cuda'))
    # y = y.to(torch.device('cuda'))
    pred = model(x)
    loss = loss_fn(pred, y)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
  print("Epoch: {} | Loss: {:.4f}".format(e,np.mean(losses)))
  best_pred = 0
  model.eval()
  with torch.no_grad():
    for (x, y) in valid_loader:
      # x = x.to(torch.device('cuda'))
      pred = torch.argmax(model(x),1)
      print("Validation Accuracy {:.4f}".format(accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y))))
      if accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y)) >= best_pred:
        best_pred = accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y))
        torch.save(model, '/content/drive/MyDrive/Colab Notebooks/best_CNN.pt')

NameError: name 'CNN' is not defined

In [15]:
import torchvision
import torch
from torch import optim
from torch import nn
from sklearn.metrics import accuracy_score
from tqdm import tqdm

model = torchvision.models.GoogLeNet()
epochs = 15
lr = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

for e in range(epochs):
  model.train()
  losses = []
  for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
    optimizer.zero_grad()
    # x = x.to(torch.device('cuda'))
    # y = y.to(torch.device('cuda'))
    pred = model(x)
    loss = loss_fn(pred.logits, y)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
  print("Epoch: {} | Loss: {:.4f}".format(e,np.mean(losses)))
  best_pred = 0
  model.eval()
  with torch.no_grad():
    for (x, y) in valid_loader:
      # x = x.to(torch.device('cuda'))
      pred = torch.argmax(model(x),1)
      print("Validation Accuracy {:.4f}".format(accuracy_score(pred, y)))
      if accuracy_score(pred, y) >= best_pred:
        best_pred = accuracy_score(pred, y)
        torch.save(model, '/content/drive/MyDrive/Colab Notebooks/best_googlenet.pt')

/usr/local/lib/python3.11/dist-packages/torchvision/models/googlenet.py:47: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(
100%|██████████| 42/42 [11:37<00:00, 16.61s/it]


Epoch: 0 | Loss: 1.1573
Validation Accuracy 0.7333


100%|██████████| 42/42 [11:38<00:00, 16.62s/it]


Epoch: 1 | Loss: 0.2938
Validation Accuracy 0.8667


100%|██████████| 42/42 [12:02<00:00, 17.21s/it]


...

Epoch: 12 | Loss: 0.1404
Validation Accuracy 0.8667


100%|██████████| 42/42 [11:33<00:00, 16.51s/it]


Epoch: 13 | Loss: 0.1278
Validation Accuracy 0.9667


100%|██████████| 42/42 [11:38<00:00, 16.63s/it]


Epoch: 14 | Loss: 0.1021
Validation Accuracy 0.8500


In [24]:
model = torch.load('/content/drive/MyDrive/Colab Notebooks/best_CNN.pt', weights_only=False)
model.eval()
with torch.no_grad():
  for (x, y) in test_loader:
    pred = torch.argmax(model(x),1)
    print("Test Accuracy {:.4f}".format(accuracy_score(pred, y)))

model = torch.load('/content/drive/MyDrive/Colab Notebooks/best_googlenet.pt', weights_only=False)
model.eval()
with torch.no_grad():
  for (x, y) in test_loader:
    pred = torch.argmax(model(x),1)
    print("Test Accuracy {:.4f}".format(accuracy_score(pred, y)))

Test Accuracy 0.8733
Test Accuracy 0.9200


In [25]:
im_transform = torchvision.transforms.Compose([torchvision.transforms.Resize((256,256)),torchvision.transforms.ToTensor(),
                                               torchvision.transforms.RandomRotation(180),
                                               torchvision.transforms.RandomHorizontalFlip(),
                                               torchvision.transforms.RandomVerticalFlip(),
                                               torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

train_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Train/Train', transform = im_transform)
train_loader = torch.utils.data.DataLoader(train_dat, batch_size = 32, shuffle = True)
valid_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Validation/Validation', transform = im_transform)
valid_loader = torch.utils.data.DataLoader(valid_dat, batch_size=len(valid_dat), shuffle = True)
test_dat = torchvision.datasets.ImageFolder('/content/drive/MyDrive/Colab Notebooks/apple_plant_disease/Test/Test', transform = im_transform)
test_loader = torch.utils.data.DataLoader(test_dat, batch_size = len(test_dat))

In [ ]:
epochs = 30
lr = 0.001
model = CNN()
# model.to(torch.device('cuda'))
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr)

for e in range(epochs):
  model.train()
  losses = []
  for i, (x, y) in tqdm(enumerate(train_loader), total=len(train_loader)):
    optimizer.zero_grad()
    pred = model(x)
    loss = loss_fn(pred, y)
    losses.append(loss.item())
    loss.backward()
    optimizer.step()
  print("Epoch: {} | Loss: {:.4f}".format(e,np.mean(losses)))
  best_pred = 0
  model.eval()
  with torch.no_grad():
    for (x, y) in valid_loader:
      pred = torch.argmax(model(x),1)
      print("Validation Accuracy {:.4f}".format(accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y))))
      if accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y)) >= best_pred:
        best_pred = accuracy_score(torch.Tensor.cpu(pred), torch.Tensor.cpu(y))
        torch.save(model, '/content/drive/MyDrive/Colab Notebooks/best_CNN_augmented.pt')

 90%|█████████ | 38/42 [05:01<00:30,  7.65s/it]